![]('./image/data_process.png')

<center>
Figure 1.数据处理
<img src="./image/data_process.png" width="70%" height="70%" />
</center>

<center>
Figure 2.textCNN原理
<img src="image/textCNN.png" width="70%" height="70%" />
</center>

1.卷积层  
卷积层输入的是一个表示句子的矩阵，维度为`n*d`，即每句话共有`n`个词，每个词有一个`d`维的词向量表示
卷积操作后再使用激活函数激活得到相应的特征`ci`，则卷可以表示为:(使用点乘来表示卷积操作)
<img src="image/1-5.png"  width="500" >
经过卷积操作之后，可以得到一个n-h+1维的向量c形如
<img src="image/2-3.png"  width="400" >
**可以使用更多高度不同的卷积核，且每个高度的卷积核多个，得到更多不同特征**

2.池化层  
**在卷积过程中使用了不同高度的卷积核，使得通过卷积层后得到的向量维度会不一致**，所以在池化层中，使用**1-Max-pooling**对每个特征向量池化成一个值，即抽取每个特征向量的最大值表示该特征，而且认为这个最大值表示的是最重要的

3.全连接层  
wx+b
假设有两层全连接层，第一层可以加上’relu’作为激活函数，第二层则使用softmax激活函数得到属于每个类的概率  
如情感分析的正负面时，第二层也可以使用sigmoid作为激活函数，然后损失函数使用对数损失函数’binary_crossentropy’

4.**在词向量构造方面可以有以下不同的方式**：

CNN-rand: 随机初始化每个单词的词向量通过后续的训练去调整。  
```
embedding = tf.get_variable(name='embedding', shape=[self.config.vocab_size, self.config.embedding_dim],initializer=self.initializer)
```
CNN-static: 使用预先训练好的词向量，如word2vec训练出来的词向量，在训练过程中不再调整该词向量。  
```
embedding = np.load(embedding_file)
embed1 = tf.constant(embedding, name='embedding')
``` 
CNN-non-static: 使用预先训练好的词向量，并在训练过程进一步进行调整。  
```
embedding = np.load(embedding_file)
embed2 = tf.Variable(embedding, name='embedding')
```
CNN-multichannel: 将static与non-static作为两通道的词向量。  
```
embedding = []
embedding.append(tf.nn.embedding_lookup(embed1, inputs))
embedding.append(tf.nn.embedding_lookup(embed2, inputs))
embed = tf.concat(embedding, axis=-1)
```

API：  
卷积过程由于只是沿着句子长度方向进行卷积，即只在一个维度卷积所以使用Conv1d。 
**Conv1d(filters, kernel_size, activation)**:
- filters: 卷积核的个数
- kernel_size: 卷积核的宽度
- activation: 卷积层使用的激活函数


池化过程使用的在一个维度上的池化，使用MaxPooling1D  
**MaxPooling1D(pool_size)**:
- pool_size: 池化窗口的大小，由于我们要将一个卷积核得到特征向量池化为1个值，所以池化窗口可以设为(句子长度-卷积核宽度+1)

 
池化过程最后还需要对每个值拼接起来，可以使用concatenate函数实现。  
**concatenate(inputs, axis)**:
- inputs: inputs为一个tensor的list，所以需要将得到1-MaxPooling得到每个值append到list中，并把该list作为inputs参数的输入。
- axis: 指定拼接的方向。

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout, Input, concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Model
import os
import tarfile
import numpy as np

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 读取数据
数据清洗：去除含有html标签的  
分词：此处为英文,不需  
去停用词：可以去除”the”、”a”等词,此处没加

In [2]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)

def read_files(filetype):
    """
    filetype: 'train' or 'test'
    return:
    all_texts: filetype数据集文本
    all_labels: filetype数据集标签
    """
    # 标签1表示正面，0表示负面
    all_labels = [1]*12500 + [0]*12500
    all_texts = []
    file_list = []
    path = r'./data/aclImdb/'
    # 读取正面文本名
    pos_path = path + filetype + '/pos/'
    for file in os.listdir(pos_path):
        file_list.append(pos_path+file)
    # 读取负面文本名
    neg_path = path + filetype + '/neg/'
    for file in os.listdir(neg_path):
        file_list.append(neg_path+file)
    # 将所有文本内容加到all_texts
    for file_name in file_list:
        with open(file_name, encoding='utf-8') as f:
            all_texts.append(rm_tags(" ".join(f.readlines())))
    return all_texts, all_labels

In [3]:
tfile = tarfile.open(r'./data/aclImdb_v1.tar.gz', 'r:gz')  # r;gz是读取gzip压缩文件
result = tfile.extractall('./data/')  # 解压缩文件到data文件夹中

In [5]:
train_texts, train_labels = read_files('train')
test_texts, test_labels = read_files('test')

### 处理成深度学习需要的数据格式
tokenizer = Tokenizer(num_words=2000)：创建词典,以空格分词  
tokenizer.fit_on_texts()：对输入的文本中的每个词编号：编号是根据词频的，词频越大，编号越小    
tokenizer.texts_to_sequences()：每一句话的文本转数字，使用每个词的编号进行编号
keras.preprocessing.sequence.pad_sequences()：将每句话补齐到相同长度,左补0

In [6]:
def preprocessing(train_texts, train_labels, test_texts, test_labels):
    tokenizer = Tokenizer(num_words=2000)  # 建立一个2000个单词的字典
    tokenizer.fit_on_texts(train_texts)
    #print(tokenizer.word_index)：得到word2id的词典
    
    # 对每一句影评文字转换为数字列表，使用每个词的编号进行编号
    x_train_seq = tokenizer.texts_to_sequences(train_texts)
    x_test_seq = tokenizer.texts_to_sequences(test_texts)
    
    x_train = sequence.pad_sequences(x_train_seq, maxlen=150)
    x_test = sequence.pad_sequences(x_test_seq, maxlen=150)
    
    y_train = np.array(train_labels)
    y_test = np.array(test_labels)
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = preprocessing(train_texts, train_labels, test_texts, test_labels)

### 建立text_cnn模型
keras.layers.embeddings.Embedding(vocab_size,embed_size)将每个词编码转换为词向量    
卷积尺寸采用2,3,4,5,每种100个卷积核  
FC1: 32  
FC2:sigmoid  

In [7]:
def text_cnn(maxlen=150, max_features=2000, embed_size=32):
    # Inputs
    comment_seq = Input(shape=[maxlen], name='x_seq')

    # Embeddings layers
    emb_comment = Embedding(max_features, embed_size)(comment_seq)

    # conv layers
    convs = []
    filter_sizes = [2, 3, 4, 5]
    for fsz in filter_sizes:
        l_conv = Conv1D(filters=100, kernel_size=fsz, activation='relu')(emb_comment)
        l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    merge = concatenate(convs, axis=1)

    out = Dropout(0.5)(merge)
    output = Dense(32, activation='relu')(out)

    output = Dense(units=1, activation='sigmoid')(output)

    model = Model([comment_seq], output)
    #adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model

### 训练模型

In [10]:
model = text_cnn()
batch_size = 128
epochs = 20
model.fit(x_train, y_train,
          validation_split=0.1,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True)

Train on 22500 samples, validate on 2500 samples
Epoch 1/20
22500/22500 [==============================] - 4s 199us/step - loss: 0.5685 - acc: 0.6836 - val_loss: 0.4979 - val_acc: 0.7556
Epoch 2/20
22500/22500 [==============================] - 3s 153us/step - loss: 0.3449 - acc: 0.8512 - val_loss: 0.3620 - val_acc: 0.8296
Epoch 3/20
22500/22500 [==============================] - 3s 154us/step - loss: 0.2875 - acc: 0.8796 - val_loss: 0.2786 - val_acc: 0.8832
Epoch 4/20
22500/22500 [==============================] - 3s 153us/step - loss: 0.2431 - acc: 0.9011 - val_loss: 0.4040 - val_acc: 0.8252
Epoch 5/20
22500/22500 [==============================] - 3s 153us/step - loss: 0.2079 - acc: 0.9180 - val_loss: 0.3534 - val_acc: 0.8524
Epoch 6/20
22500/22500 [==============================] - 3s 154us/step - loss: 0.1756 - acc: 0.9335 - val_loss: 0.2908 - val_acc: 0.8876
Epoch 7/20
22500/22500 [==============================] - 3s 153us/step - loss: 0.1450 - acc: 0.9464 - val_loss: 0.3757 - v

### 预测模型

In [11]:
scores = model.evaluate(x_test, y_test)
print('test_loss: %f, accuracy: %f' % (scores[0], scores[1]))

25000/25000 [==============================] - 3s 116us/step
test_loss: 0.693744, accuracy: 0.859160


模型训练过程batch_size设为64，epochs为20，最终可以在验证集可以得到85.9%的准确率。